In [243]:
import os
import json
import datetime
import warnings
import polars as pl
import pandas as pd
import altair as alt

from src.najdi_rok import najdi_rok
from src.pocet_stran import pocet_stran
from src.bez_bordelu import bez_bordelu
from src.alt_friendly import alt_friendly
from src.hezke_jmeno import hezke_jmeno
from src.kristi_promin import kristi_promin
from src.zjisti_vazbu import zjisti_vazbu
from src.me_to_neurazi import me_to_neurazi

with open(os.path.join('src','kredity.json'), 'r', encoding='utf-8') as kredity:
    kredity = json.loads(kredity.read())

pl.Config(tbl_rows=100)
alt.data_transformers.disable_max_rows()
alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

In [244]:
uplne_vsechno = pl.read_parquet(os.path.join("data/cnb_sloupce","008.parquet"))

In [245]:
len(uplne_vsechno)

1250624

In [246]:
df = pl.read_parquet(os.path.join("data/cnb_sloupce","leader.parquet"))
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","100.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","008.parquet")), left_on="001", right_on="001", how="left")
df = df.to_pandas()
df = df[df["leader"].str[6].isin(["a", "t"])]
df = df[~df["leader"].str[7].isin(["b", "i", "s", " "])]
df = df[(df["008"].str[15:17] == "xr") & (df["008"].str[35:38] == "cze")]
df = pl.from_pandas(df)
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","020.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","022.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","245.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","300.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","655.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","700.parquet")), left_on="001", right_on="001", how="left")
df = df.explode("022_a").filter(pl.col("022_a").is_null())
df = df.with_columns(pl.col('008').map_elements(najdi_rok, return_dtype=int).alias('rok'))
df = df.with_columns(pl.col('300_a').map_elements(pocet_stran, return_dtype=int).alias('stran'))
df = df.with_columns(pl.col('245_a').map_elements(bez_bordelu, return_dtype=str))
df = df.explode("020_q").with_columns(pl.col("020_q").map_elements(zjisti_vazbu, return_dtype=str).alias('vazba'))
df = df.explode('245_p').with_columns(pl.col('245_p').map_elements(bez_bordelu, return_dtype=str))
print(len(df))
df = df.filter(~pl.col('rok').is_null()).sort(by='rok')
df = df.filter((~pl.col("245_h").str.contains("grafika")) | pl.col("245_h").is_null())
print(len(df))

1001279
991612


In [247]:
len(df.unique(subset=['100_a','245_a']))

699002

In [248]:
rocni_pocty = df.filter(pl.col('rok').is_between(1800, 2024)).group_by('rok').len().sort('rok').rename({'len':'knih'})
rocni_pocty

rok,knih
i64,u32
1800,1
1801,22
1802,19
1803,22
1804,34
1805,23
1806,16
1807,30
1808,28


In [249]:
rocni_strany = df.filter(pl.col('rok').is_between(1800, 2024)).sort(by="rok").unique(subset=['245_a','100_a']).group_by('rok').agg(pl.col("stran").sum()).sort('rok')
rocni_strany

rok,stran
i64,i64
1800,5
1801,1789
1802,3040
1803,3613
1804,6871
1805,3166
1806,4194
1807,5828
1808,4354


In [250]:
rocni_cetba = 8 * 60 * 365
rocni_cetba

175200

In [251]:
rocni_strany.filter(pl.col('stran') <= rocni_cetba)

rok,stran
i64,i64
1800,5
1801,1789
1802,3040
1803,3613
1804,6871
1805,3166
1806,4194
1807,5828
1808,4354


In [252]:
rocni_strany.sort(by="stran",descending=True).head(5)

rok,stran
i64,i64
2008,2121418
2011,2120337
2010,2092612
2018,2091457
2009,2081351


In [253]:
rocni_strany.select(pl.col("stran")).tail(1) / rocni_cetba

stran
f64
9.183019


In [254]:
novinky = df.filter(pl.col('rok').is_between(1800, 2024)).filter(pl.col('stran') >= 50).sort('rok').unique(subset=["245_a","100_a"])
novinky = novinky.group_by('rok').len()
novinky = novinky.sort('rok').rename({'len':'novinky'})

In [255]:
novinky.sort(by="novinky",descending=True).head(5)

rok,novinky
i64,u32
2008,10590
2007,10362
2011,10284
2010,10264
2006,10154


In [256]:
celkem_vyslo = df.filter(pl.col('rok').is_between(1800, 2024)).filter(pl.col('stran') >= 50).group_by('rok').agg(pl.col('008').unique().len()).rename({'008':'celkem_vyslo'}).sort('rok')

In [257]:
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","041.parquet")), left_on="001", right_on="001", how="left")

In [258]:
nepuvodni = df.filter(pl.col('rok').is_between(1800, 2024)).filter(pl.col('stran') >= 50).explode('041_h').filter(pl.col('041_h') != 'cze').group_by('rok').agg(pl.col('008').unique().len()).rename({'008':'preklady'}).sort('rok')
nepuvodni

rok,preklady
i64,u32
1801,2
1802,3
1803,5
1804,8
1805,2
1806,3
1807,3
1808,6
1809,6


In [259]:
preklady = celkem_vyslo.join(nepuvodni, on='rok', how='left')
preklady = preklady.with_columns((pl.col('preklady') / pl.col('celkem_vyslo')).alias('podíl překladů'))
preklady = preklady.select(pl.col(['rok','podíl překladů']))
preklady

rok,podíl překladů
i64,f64
1801,0.285714
1802,0.230769
1803,0.384615
1804,0.32
1805,0.133333
1806,0.230769
1807,0.157895
1808,0.3
1809,0.315789


In [364]:
do_grafu = rocni_pocty.join(rocni_strany, on="rok", how="left")
do_grafu = do_grafu.join(novinky, on="rok", how="left")
do_grafu = do_grafu.with_columns((pl.col("stran") / (60 * 8 * 365.25)).alias("let čtení"))
# do_grafu = do_grafu.join(preklady, on='rok', how='left')
do_grafu = do_grafu.melt(id_vars="rok", value_vars=['knih','stran','let čtení','novinky'])
do_grafu = do_grafu.filter(pl.col('variable').is_in(['knih','novinky','podíl překladů']))
do_grafu = do_grafu.with_columns(pl.col('variable').replace_strict({'knih': 'všechny publikace', 'novinky': 'novinky nad 50 stran'}))
do_grafu

rok,variable,value
i64,str,f64
1800,"""všechny publikace""",1.0
1801,"""všechny publikace""",22.0
1802,"""všechny publikace""",19.0
1803,"""všechny publikace""",22.0
1804,"""všechny publikace""",34.0
1805,"""všechny publikace""",23.0
1806,"""všechny publikace""",16.0
1807,"""všechny publikace""",30.0
1808,"""všechny publikace""",28.0


In [261]:
titulek = "Vývoj knižní produkce na českém území"
podtitulek = ["Celkové počty záznamů v České národní bibliografii.","Vedle klasických knih může jít i o katalogy výstav apod."] #,"Angličtina a francouzština (téměř) zmizely jen během 2. světové války,","kdy dominovaly překlady z němčiny. Po osvobození raketově narostl podíl ruštiny."]
kredit = "zdroj dat: Česká národní bibliografie · vizualizace: iROZHLAS.cz · 2025"

In [262]:
do_grafu

rok,variable,value
i64,str,f64
1800,"""všechny publikace""",1.0
1801,"""všechny publikace""",22.0
1802,"""všechny publikace""",19.0
1803,"""všechny publikace""",22.0
1804,"""všechny publikace""",34.0
1805,"""všechny publikace""",23.0
1806,"""všechny publikace""",16.0
1807,"""všechny publikace""",30.0
1808,"""všechny publikace""",28.0


In [366]:
do_grafu_extra = []
for x in range(2024,2051):
    slovnik = { "rok": x, 'variable':'novinky nad 50 stran', 'value' : float(0)}
    do_grafu_extra.append(slovnik)
    slovnik = { "rok": x, 'variable':'všechny publikace', 'value' : float(0)}
    do_grafu_extra.append(slovnik)

In [330]:
do_grafu

rok,variable,value
i64,str,f64
1800,"""všechny publikace""",1.0
1801,"""všechny publikace""",22.0
1802,"""všechny publikace""",19.0
1803,"""všechny publikace""",22.0
1804,"""všechny publikace""",34.0
1805,"""všechny publikace""",23.0
1806,"""všechny publikace""",16.0
1807,"""všechny publikace""",30.0
1808,"""všechny publikace""",28.0


In [368]:
do_grafu = pl.concat([do_grafu, pl.DataFrame(do_grafu_extra)])

In [340]:
alt.Chart(do_grafu.head(130).to_pandas()).mark_line().encode(alt.X('rok:T'),alt.Y('value:Q'))

alt.Chart(...)

In [358]:
vyvoj_produkce = alt.Chart(
    alt_friendly(do_grafu),
    title=alt.Title('Vývoj knižní produkce na českém území'),
    width=kredity['sirka'],
    height=kredity['vyska_nizkych'] * 1,
    autosize=alt.AutoSizeParams(
        type='pad',
        contains='padding')
).mark_bar(width=1, color='#DB842F').encode(
    alt.X('rok:T', title=None, axis=alt.Axis(tickCount=5)),
    alt.Y(
        'value:Q', 
        title=None, 
        axis=alt.Axis(orient='right', domainOpacity=0, tickColor='#DCDDD6', format='s', labelExpr='split(datum.label, "k")[0] + " tis."'), 
        scale=alt.Scale(domainMin=1)),
    alt.Row('variable:N', title=None, header=alt.Header(labelAngle=0, labelAlign='left', labelAnchor='start', labelOrient='top', labelFont='Asap'), sort=['všechny publikace','novinky nad 50 stran','podíl překladů'])
).resolve_scale(
    y='independent'
).configure_view(
    stroke='transparent'
).configure_axisX(
    grid=False, domain=False
).configure_axisY(
    grid=False, domain=False
).resolve_axis(
    x="independent",
    y="independent"
).resolve_scale(
    y="shared"
)

vyvoj_produkce

alt.Chart(...)

In [266]:
vyvoj_produkce['title']['text']

'Vývoj knižní produkce na českém území'

In [402]:
do_grafu.sort("value",descending=True).head(5)

rok,variable,value
i64,str,f64
1800,"""novinky nad 50 stran""",null
2008,"""všechny publikace""",20638.0
2007,"""všechny publikace""",19768.0
2010,"""všechny publikace""",19487.0
2006,"""všechny publikace""",19178.0


In [408]:
do_grafu.filter(pl.col("value") > 0).filter(pl.col("variable") == "všechny publikace").sort("value",descending=True).tail(5)

rok,variable,value
i64,str,f64
1813,"""všechny publikace""",22.0
1802,"""všechny publikace""",19.0
1811,"""všechny publikace""",17.0
1806,"""všechny publikace""",16.0
1800,"""všechny publikace""",1.0


In [267]:
me_to_neurazi(vyvoj_produkce, kredity=kredity['default'], soubor="01_vyvoj_produkce")

<figure>
    <a href="https://data.irozhlas.cz/knihy-grafy/01_vyvoj_produkce.svg" target="_blank">
    <img src="https://data.irozhlas.cz/knihy-grafy/01_vyvoj_produkce.svg" width="100%" alt="Graf s titulkem „Vývoj knižní produkce na českém území“. Další texty by měly být čitelné ze zdrojového souboru SVG." />
    </a>
    </figure>


In [442]:
hezci_verze = alt.Chart(alt_friendly(
    do_grafu.filter(pl.col('variable') == 'všechny publikace').filter(pl.col('rok') >= 1801).with_columns(
        pl.when(pl.col('rok').is_between(
            1801,1850
        )).then(pl.lit(
            "1801-1850 | Od poličky ke knihovničce"
        )).when(pl.col('rok').is_between(
            1851,1900
        )).then(pl.lit(
            "1851-1900 | Doba národního obrození, počet knih se XXnásobí"
        )).when(pl.col('rok').is_between(
            1901,1950
        )).then(pl.lit(
            "1901-1950 | Propady za první i druhé světové války"
        )).when(pl.col('rok').is_between(
            1951,2000
        )).then(pl.lit(
            "1951-2000 | Poklidný růst až k porevolučnímu boomu"
        )).when(pl.col('rok').is_between(
            2001,2050
        )).then(pl.lit(
            "2001-dosud | Útlum po hospodářské krizi")).alias("kdy")
    )),
          title=alt.Title("Kolik knih v češtině na českém území vycházelo", 
                          subtitle=["Za rok 1806 najedeme v České národní bibliografii 16 knih,",
                                    "na to by stačila jediná polička. V roce 2008 jich bylo přes 20 tisíc.",
                                    "Takový rozdíl nejde smysluplně zobrazit na jedné škále, proto 224 let",
                                    "české literatury zachycených v datech rozdělujeme do pěti řádků.",
                                    "Každý začíná tam, kde končí předchozí."]
                         ),
          width=kredity['sirka'],
          height=kredity['vyska_nizkych'] * 0.8
).mark_bar(color='#E09DA3').encode(
    alt.X("rok:T", title=None),
    alt.Y("value", title=None, axis=alt.Axis(tickCount=3, orient="right", format="s")),
    alt.Row("kdy:N", title=None, header=alt.Header(
        labelOrient="top", labelAlign="left", labelAnchor="start", labelFont="Asap"
    ))
).resolve_scale(y="independent",x="independent").resolve_axis(x="independent").configure_view(stroke='transparent').configure_axis(grid=False, domain=True)

hezci_verze

alt.Chart(...)

In [269]:
do_grafu

rok,variable,value
i64,str,f64
1800,"""všechny publikace""",1.0
1801,"""všechny publikace""",22.0
1802,"""všechny publikace""",19.0
1803,"""všechny publikace""",22.0
1804,"""všechny publikace""",34.0
1805,"""všechny publikace""",23.0
1806,"""všechny publikace""",16.0
1807,"""všechny publikace""",30.0
1808,"""všechny publikace""",28.0
